## Argo File Loader

In [2]:
import argopy
from argopy import DataFetcher as ArgoDataFetcher
argo_loader=ArgoDataFetcher(src='gdac',ftp="/swot/SUM05/dbalwada/Argo_sync",parallel=True)

import filt_funcs as ff
import density_funcs as df
import gsw
import numpy as np

## Load by box (coordinate: N_PROF)

In [ ]:
#box=[lon_min,lon_max,lat_min,lat_max,depth_min,depth_max]
box=[25, 45, -60, -50,0,2000]

In [ ]:
ds_z=ff.get_box(box,4)

In [ ]:
ds_z = ds_z.assign_attrs({"Fetched_uri":''})

In [ ]:
ds_z

In [ ]:
ds_z.to_netcdf("202206_boxes/lon:({},{})_lat:({},{})_ds_z.nc".format(box[0],box[1],box[2],box[3]))

In [ ]:
print(ds_z.SIG0.values.min(), ds_z.SIG0.values.max())

In [ ]:
rho_grid = np.linspace(26.6, 28, 1001)

In [ ]:
ds_rho = df.interpolate2density_prof(ds_z, rho_grid)

In [ ]:
ds_rho

In [ ]:
ds_rho.to_netcdf("202206_boxes/lon:({},{})_lat:({},{})_ds_rho.nc".format(box[0],box[1],box[2],box[3]))

## Load by float (coordinate: distance)

In [ ]:
float_ID = 1901832

In [ ]:
ds_z = ff.get_float(float_ID, 4)

In [ ]:
ds_z = ds_z.assign_attrs({"Fetched_uri":''})

In [ ]:
ds_z

In [ ]:
ds_z.to_netcdf("202206_floats/float_ID:({})_ds_z.nc".format(float_ID))

In [ ]:
print(ds_z.SIG0.values.min(), ds_z.SIG0.values.max())

In [ ]:
rho_grid = np.linspace(26.4, 28, 1001)

In [ ]:
ds_rho = df.interpolate2density_prof(ds_z, rho_grid)

In [ ]:
ds_rho

In [ ]:
ds_rho.to_netcdf("202206_floats/float_ID:({})_ds_rho.nc".format(float_ID))

## Loading Boxes by Degree

In [54]:
lons = np.arange(-90,91,4)
lats = np.arange(-180,180,4)

In [78]:
box_1 = np.array([lons[0:2], lats[0:2]])
box_2 = np.array([lons[1:3], lats[1:3]])
box_3 = np.array([lons[2:4], lats[2:4]])

In [79]:
print(box_1)
print(box_2)
print(box_3)

[[ -90  -86]
 [-180 -176]]
[[ -86  -82]
 [-176 -172]]
[[ -82  -78]
 [-172 -168]]


Okay so I have a method to get the dimensions of each box and store them in an array. The next step is to write a function that gets all boxes iteratively.

In [82]:
boxes = np.array([['lon_min','lon_max'],['lat_min','lat_max']])

for n in range(0,len(lons)):
    test = np.append(boxes,[lons[n:n+2],lats[n:n+2]])

/home.ufs/amf2288/.conda/envs/argo/lib/python3.10/site-packages/numpy/core/fromnumeric.py:1859: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return asanyarray(a).ravel(order=order)


In [88]:
test = np.append(boxes,[[lons[0:2],lats[0:2]]])
test

array(['lon_min', 'lon_max', 'lat_min', 'lat_max', '-90', '-86', '-180',
       '-176'], dtype='<U21')

In [ ]:
from argopy import IndexFetcher as ArgoIndexFetcher

box_bad=[-148,-147,39,40]#, '2000-01-01', '2021-06'] # need to remove pressure poinds because the region function for this index fetcher seems a bit different.

idx = ArgoIndexFetcher(src='gdac',dataset='phy',mode='standard',
                            ftp="/swot/SUM05/dbalwada/202203-ArgoData").region(box_bad).load()
idx.index